In [ ]:
from pandas import DataFrame, read_csv, get_dummies
from pathlib import Path
from numpy import array, random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers.core import Dropout

In [ ]:
db = Path('/content/drive/MyDrive/aug+og')
selected_db = db
model_out = Path('Model')
model_name = model_out / 'ADB_9_95'

In [ ]:
ACCURACY_THRESHOLD = 0.95

y = []
x = []
f = []
e = []
for klass in list(selected_db.glob('*')):
    print(klass.name)
    
    for csv in klass.glob('*'):
        print(csv)
        d = read_csv(str(csv), header=None ).values
        if d.shape == (50,1629):
             e.append(d)
             y.append(klass.name)
        else:
             f.append(d)
        print(d.shape)

In [ ]:
with open('finalcsve.pkle','wb') as xyz:
    pickle.dump(e,xyz)
with open('finalcsvy.pkle','wb') as xyz:
    pickle.dump(y,xyz)
with open('finalcsvf.pkle','wb') as xyz:
    pickle.dump(f,xyz)

In [ ]:
y = get_dummies(DataFrame(y)).values
e = array(e)

p = random.permutation(len(y))
y = y[p]
e = e[p]

print('Data Shape Check', e.shape, y.shape)
print('Model will stop training after reacing: ', ACCURACY_THRESHOLD)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(e, y, test_size=0.2, shuffle= True)

print(f"No. of training examples: {x_train.shape[0]}")
print(f"No. of testing examples: {y_test.shape[0]}")

In [ ]:
with open('finalcsvx_train.pkle','wb') as xyz:
    pickle.dump(x_train,xyz)

In [ ]:
with open('finalcsvx_valid.pkle','wb') as xyz:
    pickle.dump(x_valid,xyz)

In [ ]:
with open('finalcsvy_train.pkle','wb') as xyz:
    pickle.dump(y_train,xyz)

In [ ]:
with open('finalcsvy_valid.pkle','wb') as xyz:
    pickle.dump(y_valid,xyz)

In [ ]:
class stop_(Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True
            
stop_training = stop_()

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=e.shape[1:]))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()


In [ ]:
a = Adam(learning_rate=0.00001)
H = model.compile(optimizer=a, loss='categorical_crossentropy', metrics=['accuracy'])
class_weight = {0: 1.,      #Address
                1: 1.,      #Movie
                2: 1.,      #Name
                3: 1.,      #Phone
                4: 1.,      #Play
                5: 1.,      #Please
                6: 1.,      #Work
                7: 1.   }  #Your
model.fit(x_train, y_train, epochs=1000, verbose=1, callbacks=[stop_training], class_weight=class_weight,validation_data=(x_valid,y_valid))

print('Saving model')
model.save(str(model_name))

In [ ]:
import matplotlib.pyplot as plt

acc = model.history['accuracy']
val_acc = model.history['val_accuracy']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()